In [19]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender



columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
len(dataset.item_id.unique()),dataset.item_id.unique().min(),dataset.item_id.unique().max()

(24507, 0, 24506)

In [20]:
from Data_manager.Build_Matrix.Data_import import build_URM_ICM,build_URM_impression,build_URM_ICM_onlyURM_item
URM,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)

In [4]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 697 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 334 ( 0.8%) Users that have less than 1 test interactions


In [7]:
import os

output_folder_path = "result_experiments"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
                "topK": Integer(5, 1000),
                "l1_ratio": Real(low = 1e-5, high = 1.0, prior = 'log-uniform'),
                "alpha": Real(low = 1e-3, high = 1.0, prior = 'uniform'),
            }

earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": URM_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                            }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = MultiThreadSLIM_SLIMElasticNetRecommender


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [8]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = False,
                    recommender_input_args_last_test=recommender_input_args
                    )

  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 194, 'l1_ratio': 7.941444844385304e-05, 'alpha': 0.1507486062928476}


100%|█████████▉| 24504/24507 [12:30<00:00, 41.57it/s]

EvaluatorHoldout: Processed 40932 (100.0%) in 39.21 sec. Users per second: 1044
SearchBayesianSkopt: New best config found. Config 0: {'topK': 194, 'l1_ratio': 7.941444844385304e-05, 'alpha': 0.1507486062928476} - results: PRECISION: 0.0398319, PRECISION_RECALL_MIN_DEN: 0.0646706, RECALL: 0.0600008, MAP: 0.0176587, MAP_MIN_DEN: 0.0282473, MRR: 0.1289016, NDCG: 0.0606409, F1: 0.0478790, HIT_RATE: 0.2890648, ARHR_ALL_HITS: 0.1502095, NOVELTY: 0.0039205, AVERAGE_POPULARITY: 0.3071896, DIVERSITY_MEAN_INTER_LIST: 0.8294793, DIVERSITY_HERFINDAHL: 0.9829459, COVERAGE_ITEM: 0.2019831, COVERAGE_ITEM_HIT: 0.0222793, ITEMS_IN_GT: 0.9948586, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.2842249, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0090906, SHANNON_ENTROPY: 7.2505388, RATIO_DIVERSITY_HERFINDAHL: 0.9832701, RATIO_DIVERSITY_GINI: 0.0180417, RATIO_SHANNON_ENTROPY: 0.5374649, RATIO_AVERAGE_POPULARITY: 6.0396188, RATIO_NOVELTY: 0.2979848, 

EvaluatorHoldout: Processed 41295 (100.0%) in 39.42 sec.

100%|█████████▉| 24504/24507 [04:06<00:00, 99.61it/s] 


...

EvaluatorHoldout: Processed 40932 (100.0%) in 23.07 sec. Users per second: 1774
SearchBayesianSkopt: Config 99 is suboptimal. Config: {'topK': 749, 'l1_ratio': 0.4094748195027777, 'alpha': 0.8023677779473986} - results: PRECISION: 0.0001661, PRECISION_RECALL_MIN_DEN: 0.0003171, RECALL: 0.0002991, MAP: 0.0000494, MAP_MIN_DEN: 0.0001079, MRR: 0.0004943, NDCG: 0.0002525, F1: 0.0002136, HIT_RATE: 0.0016613, ARHR_ALL_HITS: 0.0004943, NOVELTY: 0.0063647, AVERAGE_POPULARITY: 0.0034782, DIVERSITY_MEAN_INTER_LIST: 0.2430278, DIVERSITY_HERFINDAHL: 0.9243022, COVERAGE_ITEM: 0.0040805, COVERAGE_ITEM_HIT: 0.0007345, ITEMS_IN_GT: 0.9948586, COVERAGE_USER: 0.9832569, COVERAGE_USER_HIT: 0.0016335, USERS_IN_GT: 0.9832569, DIVERSITY_GINI: 0.0005332, SHANNON_ENTROPY: 3.9200195, RATIO_DIVERSITY_HERFINDAHL: 0.9246070, RATIO_DIVERSITY_GINI: 0.0010583, RATIO_SHANNON_ENTROPY: 0.2905815, RATIO_AVERAGE_POPULARITY: 0.0683844, RATIO_NOVELTY: 0.4837642, 



  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 245.2291
Function value obtained: -0.0000
Current minimum: -0.0224
SearchBayesianSkopt: Search complete. Best config is 40: {'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001}

SearchBayesianSkopt: Evaluation with constructor data for final test. Using best config: {'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001}


100%|█████████▉| 24504/24507 [07:18<00:00, 72.42it/s]

EvaluatorHoldout: Processed 41295 (100.0%) in 29.96 sec. Users per second: 1378
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001} - results:
CUTOFF: 10 - PRECISION: 0.0596997, PRECISION_RECALL_MIN_DEN: 0.0816009, RECALL: 0.0711107, MAP: 0.0284741, MAP_MIN_DEN: 0.0383577, MRR: 0.1843693, NDCG: 0.0828090, F1: 0.0649075, HIT_RATE: 0.3824676, ARHR_ALL_HITS: 0.2274822, NOVELTY: 0.0043316, AVERAGE_POPULARITY: 0.1960182, DIVERSITY_MEAN_INTER_LIST: 0.9451820, DIVERSITY_HERFINDAHL: 0.9945159, COVERAGE_ITEM: 0.1833354, COVERAGE_ITEM_HIT: 0.0541478, ITEMS_IN_GT: 0.9984494, COVERAGE_USER: 0.9919767, COVERAGE_USER_HIT: 0.3793990, USERS_IN_GT: 0.9919767, DIVERSITY_GINI: 0.0238329, SHANNON_ENTROPY: 9.1306331, RATIO_DIVERSITY_HERFINDAHL: 0.9948439, RATIO_DIVERSITY_GINI: 0.0473000, RATIO_SHANNON_ENTROPY: 0.6768317, RATIO_AVERAGE_POPULARITY: 3.8538916, RATIO_NOVELTY: 0.3292346, 


Se

In [7]:
from Recommenders.DataIO import DataIO
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender
import pandas as pd
recommender_class = MultiThreadSLIM_SLIMElasticNetRecommender

output_folder_path = "result_experiments"
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
recommender_class.load_model(MultiThreadSLIM_SLIMElasticNetRecommender,"/Users/loren/Documents/GitHub/RecSys/result_experiments_MultiThreadSLIM_SLIM_S_ElasticNetRecommender/result_experiments_MultiThreadSLIM_SLIMElasticNetRecommender_best_model_last.zip")
time_df = search_metadata["time_df"]

print('mean-time:')
print(time_df['train'].mean())


result_on_test_df = search_metadata["result_on_test_df"]
result_best_on_test = search_metadata["result_on_last"]
print('BEST_HYPERPARAMETERS: ')
print(search_metadata["hyperparameters_best"])
print(pd.DataFrame(result_on_test_df))
print(result_best_on_test)

TypeError: _print() missing 1 required positional argument: 'string'

In [24]:
recommender_class = MultiThreadSLIM_SLIMElasticNetRecommender(URM)
output_folder_path = "/Users/loren/Documents/GitHub/RecSys/result_experiments/"
recommender_class.load_model(output_folder_path,file_name = "SLIMElasticNetRecommender_best_model_last.zip" )


SLIMElasticNetRecommender: Loading model from file '/Users/loren/Documents/GitHub/RecSys/result_experiments/SLIMElasticNetRecommender_best_model_last.zip'
SLIMElasticNetRecommender: Loading complete


In [25]:
def prepare_submission(urm_train: sp.csr_matrix, recommender: object, test_users: pd.DataFrame):    
    recommendation_length = 10
    submission = []
    for user_id in test_users.user_id:
        recommendations = recommender.recommend(user_id_array=user_id,
                                                cutoff=recommendation_length,
                                                remove_seen_flag=True)
        
        submission.append((user_id, [item_id for item_id in recommendations]))
        
    return submission

In [26]:
def write_submission(submissions):
    with open("./MultiThreadSLIM_SLIMElasticNetRecommender.csv", "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")

In [ ]:
submission = prepare_submission(URM, recommender_class,test_users)
write_submission(submission)